# ROB 204 - Gaze Lab

In [5]:
import cv2
from picamera2 import Picamera2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import threading
import ipywidgets as widgets

import numpy as np
from IPython.display import clear_output
from IPython import display
import time
import sys, math

# FUNCTION GENERATED WITH CHATGPT -- REVISE!!!
def rotation_matrix_to_euler_angles(R):
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6
    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0
    return np.array([x, y, z])
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=False,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

tracking = None
tracking_run = False

tracking_status = widgets.HTML(value="Tracking not started")

file = open("images/placeholder.jpg", "rb")
image = file.read()
tracking_image  = widgets.Image(value=image,format='jpeg',width=640,height=480)

tracking_output = widgets.HTML(value="🙈")
tracking_fps = widgets.HTML(value="FPS: 0")

disp_container = widgets.VBox([tracking_status, tracking_image, tracking_output, tracking_fps])
output = widgets.Output()

def tracking_function(tracking_status, tracking_image, tracking_output, tracking_fps):
    tracking_status.value = "Tracking starting..."
    with Picamera2() as picam:
        picam.configure(picam.create_video_configuration(main={"format": 'RGB888', "size": (1280, 720)}))
        picam.start()
        tracking_status.value = "Tracking started."
        while tracking_run:
            start_time = time.time()
            image = picam.capture_array("main")
            image = cv2.flip(image, 1)
    
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
            detection_result = detector.detect(mp_image)

            resize_frame = cv2.resize(image, (0, 0), fx = 0.25, fy = 0.25)
            resize_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            _,ret_array = cv2.imencode('.jpg', resize_frame)
            tracking_image.value = ret_array
            
            try:
                transformation_matrix = detection_result.facial_transformation_matrixes[0]
                rotation_angles = rotation_matrix_to_euler_angles(transformation_matrix)
                yaw_angle = np.degrees(rotation_angles[1])
                tracking_output.value = str(yaw_angle)
    
            except Exception as e:
                tracking_output.value = "🙈"
                pass

            tracking_fps.value = "FPS: " + str(1.0 / (time.time() - start_time))

def start_tracking(b):
    global tracking, tracking_run, disp_container
    with output:
        if tracking is None or not tracking.is_alive():
            tracking_run = True
            tracking = threading.Thread(target=tracking_function, args=(tracking_status, tracking_image, tracking_output, tracking_fps, ))
            tracking.start()

def stop_tracking(b):
    global tracking, tracking_run
    with output:
        if tracking is not None and tracking.is_alive():
            tracking_run = False
            tracking_status.value = "Tracking stopped."


start_btn = widgets.Button(description="Start Tracking", button_style='success')
start_btn.on_click(start_tracking)

stop_btn = widgets.Button(description="Stop Tracking", button_style='danger')
stop_btn.on_click(stop_tracking)

toolbar = widgets.HBox([start_btn, stop_btn])

display.display(widgets.VBox([toolbar, disp_container]), output)

W0000 00:00:1721313324.946519    3136 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1721313324.954366    3369 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1721313324.973469    3371 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Output()